# **Analyse rapide de methadonnées**

L'objectif de ce notbook est de faire une breve analyse de methédonnée disponibles sur l'ensemble des images de radiographie de torse. 

<html>
<head>
    <title>Todo List</title>
    <style>
        .todo {
            color: rgb(250, 154, 64);
        }
    </style>
</head>

<body>
    <h1>Todo List</h1>
    <ul>
        <li class="todo">Introduction</li>
        <li class="todo">Conclusion</li>
        <li class="todo">New Data for pre-processing of images</li>
    </ul>
</body>

</html>


In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', append=True, category=UserWarning)

In [2]:
df = pd.read_csv('DATASET/metadata.csv')

<p style="color:rgb(250, 154, 64);">
      On regarde qq staistique sur les jeux de données </p>

# Analyse des Métadonnées et Prétraitement par Reweighting

Dans ce notebook, nous allons analyser les métadonnées d'un jeu de données médical où :
- Les **colonnes sensibles** sont `Patient Age` et `Patient Gender`.
- La **variable cible** est `Finding Labels`.  
  Si la valeur est `"No Finding"`, cela signifie qu'il n'y a **aucune maladie** détectée.  
  Dans le cas contraire, la chaîne contient une ou plusieurs maladies (séparées par `|`).

L'objectif est de :

1. Préparer les données en créant une variable binaire `disease` (0 pour *No Finding*, 1 pour présence de maladie).
2. Réaliser une analyse univariée et bivariée en utilisant Plotly Express pour visualiser les distributions et relations.
3. Calculer des poids pour chaque échantillon via un **reweighting** (pour compenser un éventuel déséquilibre de classes) et stocker ces poids dans la colonne `WEIGHTS` (déjà initialisés à 1.0).

"""


In [3]:
# Affichage d'un aperçu des données
print("Aperçu des premières lignes:")
display(df.head())

# Vérifier la présence des colonnes sensibles et de la colonne cible
print("Colonnes du DataFrame:", df.columns.tolist())

Aperçu des premières lignes:


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],WEIGHTS
0,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,1
1,00000022_000.png,No Finding,0,22,48,M,PA,2048,2500,0.171,0.171,1
2,00000022_001.png,Fibrosis,1,22,49,M,PA,2500,2048,0.168,0.168,1
3,00000033_000.png,Atelectasis|Cardiomegaly|Fibrosis,0,33,72,F,PA,2992,2745,0.143,0.143,1
4,00000091_000.png,No Finding,0,91,69,M,PA,2834,2849,0.143,0.143,1


Colonnes du DataFrame: ['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID', 'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'WEIGHTS']


## 1. Préparation des données

In [4]:
# Création de la variable cible binaire
#  - Si Finding Labels == "No Finding" => 0 (aucune maladie)
#  - Sinon => 1 (présence d'une ou plusieurs maladies)
df['disease'] = df['Finding Labels'].apply(lambda x: 0 if x == "No Finding" else 1)

In [5]:
# Correction des âges incorrects : remplacement des âges > 140 par la médiane des âges valides
valid_age_mask = df['Patient Age'] <= 140
median_age = df.loc[valid_age_mask, 'Patient Age'].median()
df.loc[~valid_age_mask, 'Patient Age'] = median_age

In [6]:
# Calcul de la distribution des malades/sains
distribution = df['disease'].value_counts().reset_index()
distribution.columns = ['disease', 'count']
distribution['disease'] = distribution['disease'].map({0: 'Sain', 1: 'Malade'})

# Affichage de la distribution avec un pie chart
fig_pie = px.pie(distribution, values='count', names='disease', 
                 title="Répartition des malades/sains parmi le jeu de données",
                 color_discrete_sequence=px.colors.qualitative.Pastel)
fig_pie.show()

## 2. Analyse

### 2.1 Analyse Univariée

In [7]:
# 1. Distribution de Patient Age
fig_age = px.histogram(df, 
                       x='Patient Age', 
                       title="Distribution de l'âge des patients",
                       nbins=40,
                       color_discrete_sequence=['skyblue'],
                       opacity=0.75,
                       labels={'Patient Age': 'Âge du patient'},
                       )
fig_age.show()

In [8]:
#2 Distribution du Genre

gender_distribution = df['Patient Gender'].value_counts().reset_index()
gender_distribution.columns = ['Patient Gender', 'count']

# Affichage de la distribution avec un pie chart
fig_pie_gender = px.pie(gender_distribution, values='count', names='Patient Gender',
                        title="Répartition du Genre ",
                        color_discrete_sequence=px.colors.qualitative.Pastel)
fig_pie_gender.show()

### 2.2 Analyse bivariée

In [9]:
# 1. Boxplot de Patient Age en fonction de la variable cible
fig_box_age = px.box(df, x='disease', y='Patient Age', 
                     title="Âge des patients selon la présence ou l'absence de maladie",
                     labels={'disease': 'Présence de maladie (0 = Sain, 1 = Malade)', 'Patient Age': 'Âge du patient'})
fig_box_age.show()


In [10]:
# Calcul du taux de malades par genre
df_gender_disease = df.groupby('Patient Gender')['disease'].value_counts(normalize=True).unstack().reset_index()

# Graphique à barres comparant le taux de malades chez les hommes et les femmes
fig_bar = px.bar(df_gender_disease, 
                 x='Patient Gender', 
                 y=[0, 1],  # 0 correspond aux sains, 1 correspond aux malades
                 labels={'Patient Gender': 'Genre', 'value': 'Taux de Malade'}, 
                 title="Comparaison du Taux de Malades entre Hommes et Femmes",
                 barmode='group',  
                 text_auto=True)  

fig_bar.show()

On remarque qu'il y a une légère différence de répartition des non-malades entre femmes et hommes .

## Repondération

In [11]:
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import LabelEncoder

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [ ]:
# On encode et copie le dataframe pour pouvoir utiliser aif360
dfr = df.copy()
label_encoder = LabelEncoder()
for column in dfr.select_dtypes(include=['object']).columns:
    dfr[column] = label_encoder.fit_transform(dfr[column])

### Par le genre

In [14]:
dataset_gender = StandardDataset(
    dfr,
    label_name="disease",  
    favorable_classes=[0],  
    protected_attribute_names=["Patient Gender"],  
    privileged_classes=[[1]],  
)
reweigher_gender = Reweighing(
    unprivileged_groups=[{'Patient Gender': 0}],  # Femmes
    privileged_groups=[{'Patient Gender': 1}]       # Hommes
)
dataset_transf_gender = reweigher_gender.fit_transform(dataset_gender)
dfr['WEIGHTS_GENDER'] = dataset_transf_gender.instance_weights

In [18]:
print(dfr['WEIGHTS_GENDER'].head(10))

0    0.979018
1    0.979018
2    1.027359
3    0.971271
4    0.979018
5    1.027359
6    1.027359
7    1.027359
8    1.027359
9    1.027359
Name: WEIGHTS_GENDER, dtype: float64


### Par l'âge

In [15]:
# --- 2. Reweighting par l'âge ---
# Créer une variable binaire "Age Privileged" en fonction du label 'disease' et des quartiles d'âge observés
def age_privileged(row):
    if row['disease'] == 0:
        # Pour les patients sans maladie, le groupe privilégié est défini pour les âges entre 34 et 59
        return int(34 <= row['Patient Age'] <= 59)
    else:
        # Pour les patients malades, le groupe privilégié est pour les âges entre 42 et 62
        return int(42 <= row['Patient Age'] <= 62)

dfr['Age Privileged'] = dfr.apply(age_privileged, axis=1)

dataset_age = StandardDataset(
    dfr,
    label_name="disease",  
    favorable_classes=[0],  
    protected_attribute_names=["Age Privileged"],  
    privileged_classes=[[1]],  # Groupe privilégié : Age dans l'intervalle défini
)
reweigher_age = Reweighing(
    unprivileged_groups=[{'Age Privileged': 0}],
    privileged_groups=[{'Age Privileged': 1}]
)
dataset_transf_age = reweigher_age.fit_transform(dataset_age)
dfr['WEIGHTS_AGE'] = dataset_transf_age.instance_weights

In [ ]:
print(dfr['WEIGHTS_AGE'].head(10))

### Par le genre ET l'âge

In [16]:
# --- 3. Reweighting combiné (Genre + Âge) ---
# Définir une variable "Combined Privileged" qui vaut 1 si le patient a à la fois un genre et un âge privilégié, sinon 0
dfr['Combined Privileged'] = ((dfr['Patient Gender'] == 1) & (dfr['Age Privileged'] == 1)).astype(int)

dataset_combined = StandardDataset(
    dfr,
    label_name="disease",  
    favorable_classes=[0],  
    protected_attribute_names=["Combined Privileged"],  
    privileged_classes=[[1]],  # Privé si les deux conditions sont remplies
)
reweigher_combined = Reweighing(
    unprivileged_groups=[{'Combined Privileged': 0}],
    privileged_groups=[{'Combined Privileged': 1}]
)
dataset_transf_combined = reweigher_combined.fit_transform(dataset_combined)
dfr['WEIGHTS_COMBINED'] = dataset_transf_combined.instance_weights

In [19]:
# Vérification rapide : afficher les 10 premières lignes avec les 3 colonnes de poids
print(dfr[['WEIGHTS_GENDER', 'WEIGHTS_AGE', 'WEIGHTS_COMBINED']].head(10))

   WEIGHTS_GENDER  WEIGHTS_AGE  WEIGHTS_COMBINED
0        0.979018     0.995731          1.005081
1        0.979018     1.003901          0.985541
2        1.027359     0.995195          1.018568
3        0.971271     1.005356          0.993758
4        0.979018     0.995731          1.005081
5        1.027359     1.005356          0.993758
6        1.027359     1.005356          0.993758
7        1.027359     1.005356          0.993758
8        1.027359     1.005356          0.993758
9        1.027359     1.005356          0.993758
